# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We repeat our definition of the `Spouse` `Candidate` subclass:

In [18]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We reload the probabilistic training labels:

In [21]:
from snorkel.annotations import load_marginals

dev_marginals = load_marginals(session, split=1)

We also reload the candidates:

In [22]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

Finally, we load gold labels for evaluation:

In [23]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [24]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      10,
    'dropout':       0.25,
    'seed':          1701
}

lstm = LSTM(n_threads=None)
lstm.train(dev_cands, dev_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=2180  #epochs=10  batch size=64
[LSTM] Epoch 1 (10.50s)	Average loss=0.625458	Dev F1=1.89
[LSTM] Epoch 2 (26.66s)	Average loss=0.608130	Dev F1=19.53
[LSTM] Epoch 3 (43.23s)	Average loss=0.596617	Dev F1=9.76
[LSTM] Epoch 4 (58.12s)	Average loss=0.593150	Dev F1=28.64
[LSTM] Epoch 5 (72.87s)	Average loss=0.589633	Dev F1=30.92
[LSTM] Epoch 6 (87.56s)	Average loss=0.590661	Dev F1=30.34
[LSTM] Epoch 7 (101.73s)	Average loss=0.586047	Dev F1=34.92
[LSTM] Epoch 8 (116.25s)	Average loss=0.584985	Dev F1=32.67
[LSTM] Epoch 9 (130.08s)	Average loss=0.586709	Dev F1=35.06
[LSTM] Model saved as <LSTM>
[LSTM] Epoch 10 (145.64s)	Average loss=0.587926	Dev F1=33.13
[LSTM] Training done (151.43s)
[LSTM] Loaded model <LSTM>


Now, we get the precision, recall, and F1 score from the discriminative model:

In [ ]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [ ]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [ ]:
lstm.save_marginals(session, test_cands)

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!